In [269]:
# Config
# Tables
import pandas as pd
# Charting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Math
import numpy as np
# Tools
import os
from datetime import datetime
# Data Processing
from sklearn.preprocessing import minmax_scale

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [223]:
def load_combustible():
    # Leer el archivo crudo
    path = './data/Combustible-PIB/'
    df = pd.read_excel(path+'Importaciones_Crudo_6.xls')

    # Quitar header y footer
    df = df[5:-3].reset_index(drop=True)

    # Llenar las columnas y eliminarla como fila
    df.columns = df.loc[0].ffill()
    df = df[1:]

    # Crear Multi-level Columns
    ## Crear Tuple con las columnas y subcolumnas
    columns = [('Año/Mes','Año/Mes')]
    for col in df.columns.unique()[1:]:
        for factor in list(set(df.loc[1][1:])):
            columns.append((col, factor))
    ## Poner Multi-level columns y eliminarlas como fila
    df.columns = pd.MultiIndex.from_tuples(tuple(columns))
    df = df[1:].reset_index(drop=True)

    # Guardar variable año
    primer_ano = df.loc[0][0]

    # Solo dejar las filas con meses (no totales, etc.)
    ## Encontrar todos los meses
    filas_anos = list(df['Año/Mes'].value_counts()[:12].reset_index()['Año/Mes'].values)
    ## Agregar columna que diga si es mes o no
    df['Es_ano'] = np.where(
        df['Año/Mes'].isin(filas_anos), 
        False, 
        True
    )
    ## Solo dejar las filas con meses
    df = df[~df['Es_ano']].reset_index(drop=True)
    
    # Agregar columna fecha
    num_meses = df.shape[0]
    rng = pd.date_range(f'{primer_ano}-01-01', periods=num_meses, freq='MS')
    df.index = rng
    df = df.drop(columns=['Año/Mes','Es_ano', 'Total'], level=0)

    # Save
    df.to_csv('test.csv')

    return df
combustibles = load_combustible()

In [226]:
def solo_column_level_1(df, column):
    df = df.loc[:, (slice(None), column)].droplevel(1, axis=1)
    df = df.reset_index()
    df = pd.melt(df, id_vars='index', value_vars=df.columns[1:])
    df.columns = ['Fecha','Combustible', column]
    return df

volumen = solo_column_level_1(combustibles, 'Volumen')
fig = px.bar(volumen, x='Fecha', y='Volumen', color='Combustible', title = f'Volumen importado por tipo de Combustible').show()


In [289]:
def load_imae():
    df = pd.read_excel(path+'imae.xlsx')[5:].reset_index(drop=True)

    # Llenar las columnas y eliminarla como fila
    df.columns = df.loc[0]
    df = df[1:].reset_index(drop=True)

    # Seleccionar IMAE
    df = df.iloc[:, 1:3][2:-1].reset_index(drop=True)
    # Rename columns
    df.columns = ['Mes', 'Indice']
    # Drop Totales de Anos
    df = df.dropna(subset=['Mes']).reset_index(drop=True)

    # Agregar Fecha y eliminar Mes
    primer_ano = 2007
    num_meses = df.shape[0]
    df['Fecha'] = pd.date_range(f'{primer_ano}-01-01', periods=num_meses, freq='MS')
    df = df[['Fecha','Indice']]
    df.Indice = df.Indice.astype(float)

    df.to_csv('test.csv')
    
    px.line(df,x='Fecha', y='Indice',title='IMAE').show()

    return df
    
imae = load_imae()

In [290]:
def juntar_variables(volumen, imae):
    earliest = max(min(volumen.Fecha), min(imae.Fecha))
    latest = min(max(volumen.Fecha), max(imae.Fecha))
    
    imae = imae[(imae['Fecha'] >= earliest) & (imae['Fecha'] <= latest)]
    volumen = volumen[(volumen['Fecha'] >= earliest) & (volumen['Fecha'] <= latest)]
    
    inner_join = pd.merge(volumen, 
                      imae, 
                      on ='Fecha', 
                      how ='inner')

    inner_join['Multiplicacion'] = inner_join['Volumen'] * inner_join['Indice']
    inner_join['Multiplicacion'] = minmax_scale(inner_join['Multiplicacion'])

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Scatter(
            name='IMAE',
            x=imae.Fecha,
            y=imae.Indice
        ),secondary_y=True,)

    for combustible in volumen.Combustible.unique():
        df = volumen[volumen['Combustible'] == combustible]
        fig.add_trace(
            go.Bar(
                name = combustible,
                x=df.Fecha,
                y=df.Volumen,
            ),secondary_y=False,)
    fig.update_layout(title_text='Importaciones de Combustibles vs. IMAE', barmode='stack')
    fig.show()

    return inner_join

juntos = juntar_variables(volumen, imae)

In [307]:
px.scatter(juntos[juntos.Combustible=='Petroleo Crudo'],x='Volumen',y='Indice', color='Indice', title='Volumen del Petroleo crudo importado vs. IMAE').show()

In [306]:
def vs_3d_chart(juntos, combustible):
    df = juntos[juntos.Combustible == combustible]
    fig = px.scatter_3d(
        df,
        x = 'Fecha',
        y = 'Volumen',
        z = 'Indice',
        color = 'Indice',
        title = f'Relacion de {combustible} con el IMAE',
        opacity=0.7,
        size_max=10,
    )
    
    fig.update_layout(
        margin=dict(l=0, r=0, b=10, t=0),
        scene={
        'yaxis': {'autorange': 'reversed'}, # reverse automatically
        'xaxis': {'autorange': 'reversed'}, # reverse automatically

        }
        )
    fig.show()

vs_3d_chart(juntos, 'Petroleo Crudo')

In [302]:
vs_3d_chart(juntos, 'Gasolina')

In [294]:
px.scatter(juntos[juntos.Combustible=='Petroleo Crudo'],x='Fecha',y='Volumen', color='Indice').show()